### Content Based Recommendations

In the previous notebook, you were introduced to a way of making recommendations using collaborative filtering.  However, using this technique there are a large number of users who were left without any recommendations at all.  Other users were left with fewer than the ten recommendations that were set up by our function to retrieve....

In order to help these users out, let's try another technique: **content based** recommendations. Let's start off where we were in the previous notebook.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from IPython.display import HTML
#import progressbar
#import tests as t
import pickle


%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']


all_recs = pickle.load(open("all_recs.p", "rb"))

In [2]:
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [3]:
reviews.head()

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,1,68646,10,1381620027,2013-10-12 23:20:27,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,1,113277,10,1379466669,2013-09-18 01:11:09,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,2,422720,8,1412178746,2014-10-01 15:52:26,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,2,454876,8,1394818630,2014-03-14 17:37:10,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,2,790636,7,1389963947,2014-01-17 13:05:47,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [ ]:
all_recs

### Datasets

From the above, you now have access to three important items that you will be using throughout the rest of this notebook.  

`a.` **movies** - a dataframe of all of the movies in the dataset along with other content related information about the movies (genre and date)


`b.` **reviews** - this was the main dataframe used before for collaborative filtering, as it contains all of the interactions between users and movies.


`c.` **all_recs** - a dictionary where each key is a user, and the value is a list of movie recommendations based on collaborative filtering

For the individuals in **all_recs** who did receive 10 recommendations using collaborative filtering, we don't really need to worry about them.  However, there were a number of individuals in our dataset who did not receive any recommendations.

-----

`1.` Let's start with finding all of the users in our dataset who didn't get all 10 ratings we would have liked them to have using collaborative filtering.  

In [47]:
# Store user ids who have all their recommendations in this (10 or more)
users_with_all_recs = [iD for iD in all_recs.keys() if len(all_recs[iD])>=10] 

# Store users who still need recommendations here
#users_who_need_recs = [iD for iD in all_recs.keys() if len(all_recs[iD]) < 10] # coz all_recs dont have all users
users_who_need_recs = list(set(reviews.user_id.unique())- set(users_with_all_recs))

In [5]:
len(users_with_all_recs), len(users_who_need_recs)

(22187, 31781)

In [ ]:
# A quick test
assert len(users_with_all_recs) == 22187
print("That's right there were still another 31781 users who needed recommendations when we only used collaborative filtering!")

### Content Based Recommendations

You will be doing a bit of a mix of content and collaborative filtering to make recommendations for the users this time.  This will allow you to obtain recommendations in many cases where we didn't make recommendations earlier.     

`2.` Before finding recommendations, rank the user's ratings from highest to lowest. You will move through the movies in this order looking for other similar movies.

In [6]:
rankedDf = reviews.sort_values(by=["user_id", "rating"], ascending=False)
rankedDf[rankedDf.user_id == 19753]

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
260610,19753,1024648,9,1463172041,2016-05-13 20:40:41,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
260611,19753,1655441,8,1463264355,2016-05-14 22:19:15,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
260612,19753,2452386,8,1466803243,2016-06-24 21:20:43,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
260614,19753,3797868,7,1463280245,2016-05-15 02:44:05,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
260613,19753,2702724,5,1463330898,2016-05-15 16:48:18,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


### Similarities

We have talked in our first Lecture at ITI about similarity metrics and the difference between dot product vs. cosine similarity vs euclidean distance. Here we will apply what we have learnt. 


In many cases, it turns out that one of the fastest ways we can find out how similar items are to one another (when our matrix isn't totally sparse like it was in the earlier section) is by simply using matrix multiplication.

For us to pull out a matrix that describes the movies in our dataframe in terms of content, we might just use the indicator variables related to **year** and **genre** for our movies.  

Then we can obtain a matrix of how similar movies are to one another by taking the dot product of this matrix with itself.  

We can perform the dot product on a matrix of movies with content characteristics to provide a movie by movie matrix where each cell is an indication of how similar two movies are to one another.  




`3.` Create a numpy array that is a matrix of indicator variables related to year (by century) and movie genres by movie.  Perform the dot product of this matrix with itself (transposed) to obtain a similarity matrix of each movie with every other movie.  The final matrix should be 31245 x 31245.

In [22]:
# Subset so movie_content is only using the dummy variables for each genre and the 3 century based year dummy columns

movie_content = movies.iloc[:, 4:]
# Take the dot product to obtain a movie x movie matrix of similarities

similarities = np.dot(movie_content, movie_content.T).astype('int32')
np.fill_diagonal(similarities, 0)
similarities

array([[0, 3, 3, ..., 0, 0, 0],
       [3, 0, 3, ..., 0, 0, 0],
       [3, 3, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 2, 1],
       [0, 0, 0, ..., 2, 0, 1],
       [0, 0, 0, ..., 1, 1, 0]])

In [8]:
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
# create checks for the dot product matrix
assert dot_prod_movies.shape[0] == 31245
assert dot_prod_movies.shape[1] == 31245
assert dot_prod_movies[0, 0] == np.max(dot_prod_movies[0])
print("Looks like you passed all of the tests.  Though they weren't very robust - if you want to write some of your own, I won't complain!")

### For Each User...


Now you have a matrix where each user has their ratings ordered.  You also have a second matrix where movies are each axis, and the matrix entries are larger where the two movies are more similar and smaller where the two movies are dissimilar.  This matrix is a measure of content similarity. Therefore, it is time to get to the fun part.

For each user, we will perform the following:

    i. For each movie, find the movies that are most similar that the user hasn't seen.

    ii. Continue through the available, rated movies until 10 recommendations or until there are no additional movies.

As a final note, you may need to adjust the criteria for 'most similar' to obtain 10 recommendations.  As a first pass, I used only movies with the highest possible similarity to one another as similar enough to add as a recommendation.

`3.` In the cell below, complete each of the functions needed for making content based recommendations.

In [138]:
find_similar_movies(8)

array(['À propos de Nice (1930)', 'Close-Up on Kuwait (1961)',
       'La Soufrière - Warten auf eine unausweichliche Katastrophe (1977)',
       'Chelsea Bridge Boys (1965)', 'Thomas Bennett (2013)',
       'Annabelle Serpentine Dance (1895)',
       'Roundhay Garden Scene (1888)', 'The Arrival of a Train (1896)',
       'La sortie des usines Lumière (1895)',
       'Llandudno Happy Valley and Minstrel Show (1898)'], dtype=object)

In [137]:
def find_similar_movies(movieId):
    '''
    INPUT
    movie_id - a movie_id 
    OUTPUT
    similar_movies - an array of the most similar movies by title
    '''
    # find the row of each movie id
    idxMovie = movies[movies.movie_id == movieId].index
    
    # find the most similar movie indices - to start I said they need to be the same for all content
    indexes = np.argsort(similarities[idxMovie])[:, -10:]
    
    # pull the movie titles based on the indices
    similar_movies = movies.iloc[np.squeeze(indexes)]['movie'].values
    
    return similar_movies
    

def get_movie_names(movie_ids):
    '''
    INPUT
    movie_ids - a list of movie_ids
    OUTPUT
    movies - a list of movie names associated with the movie_ids
    
    '''
    movie_lst = movies.iloc[movie_ids]['movie'].values
    # return a list with movie names for the list of input movie IDs
   
    return movie_lst


def make_recs():
    '''
    INPUT
    None
    OUTPUT
    recs - a dictionary with keys of the user and values of the recommendations
    '''
    # Create dictionary to return with users and ratings
    
    userRatings = {}
    
    # For each user
    for user in users_who_need_recs :

        # Pull only the reviews the user has seen
        
        # Look at each of the movies (highest ranked first), 
        highestMovieId = rankedDf[rankedDf.user_id ==user].movie_id.values[0]
        
        # pull the movies the user hasn't seen that are most similar
        movieLst = find_similar_movies(highestMovieId)
        # These will be the recommendations - continue until 10 recs 
        # or you have depleted the movie list for the user


        userRatings[user] = movieLst

    
    return userRatings

In [112]:
recs = make_recs()

In [133]:
recs

{1: array(['The Sting (1973)', "Killer's Kiss (1955)", 'Hanyo (1960)',
        'Raising Arizona (1987)', 'The Quiller Memorandum (1966)',
        'House of Bamboo (1955)', 'The Green Buddha (1955)',
        'The Seven-Ups (1973)', 'Objectif: 500 millions (1966)',
        'The Rainmaker (1997)'], dtype=object),
 4: array(['Independence Day: Resurgence (2016)', '2012 (2009)',
        'The Day After Tomorrow (2004)', 'Outlander (2008)',
        'Star Trek: Nemesis (2002)', 'X-Men: Days of Future Past (2014)',
        'Star Trek Beyond (2016)', 'After Earth (2013)',
        'Hunting the Phantom (2014)', 'Pacific Rim (2013)'], dtype=object),
 5: array(['Robin Hood (1991)', 'Against All Flags (1952)',
        'Mere Jeevan Saathi (1972)', 'Quigley Down Under (1990)',
        'La bataille de San Sebastian (1968)',
        'The Count of Monte Cristo (1934)',
        'The Poseidon Adventure (1972)', 'Azhakan (1991)',
        'Race for the Yankee Zephyr (1981)', 'Farewell to the King (1989)'],
  

### How Did We Do?

Now that you have made the recommendations, how did we do in providing everyone with a set of recommendations?

`4.` Use the cells below to see how many individuals you were able to make recommendations for, as well as explore characteristics about individuals for whom you were not able to make recommendations.  

In [131]:
# Explore recommendations


In [132]:
# Some characteristics of my content based recommendations
users_without_all_recs = users_who_need_recs#store user ids without recs
users_with_all_recs = users_with_all_recs# users with all recs here
no_recs = set(reviews.user_id.unique()) - set(users_who_need_recs + users_with_all_recs)# users with no recommendations

print("There were {} users without all 10 recommendations we would have liked to have.".format(len(users_without_all_recs)))
print("There were {} users with all 10 recommendations we would like them to have.".format(len(users_with_all_recs)))
print("There were {} users with no recommendations at all!".format(len(no_recs)))

There were 31781 users without all 10 recommendations we would have liked to have.
There were 22187 users with all 10 recommendations we would like them to have.
There were 0 users with no recommendations at all!
